In [ ]:
order=10

In [1]:
# import libraries
import glob                               #can be used to find the target files
import matplotlib.pyplot as plt
import nibabel as nib                   # to read and write neuron imaging data 
import numpy as np                        # for matrix calculating
import subprocess
from PIL import Image 
import os,shutil

from scipy import signal
from skimage import color, img_as_ubyte, io

In [2]:
file_script = 'results_script1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

In [3]:
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/BVECTOR.scheme','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_FA.nii.gz','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_V3.nii.gz','results_script1')

CMD = 'shutil.copy(“fsltensor_V1.nii.gz”,”results_script1”)'
subprocess.call(CMD,shell=True)


In [4]:
cd results_script1

/home/zhiwei/Desktop/semester-projects/results_script1


## generate original tractography and get fib_original, hdr_original

In [5]:
data = nib.load('fsltensor_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask.nii')

In [7]:
CMD='tckgen fsltensor_V3.nii.gz origin0.015.tck -algorithm FACT -seed_image mask.nii -mask mask.nii -select 3000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [8]:
CMD = 'TractConverter.py -i origin0.015.tck -o origin0.015.trk -a fsltensor_FA.nii.gz'
subprocess.call(CMD,shell=True)

0

In [9]:
fib_original,hdr_original=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/origin0.015.trk')

## generate synthetic CLARITY data

In [10]:
CMD='tckmap origin0.015.tck syn_CLARITY.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0


## calculate structure tensor

In [11]:
def doggen(sigma=None, X=None, Y=None, Z=None):
    halfsize = np.ceil(3 * np.max(sigma))       #3*the maximum number of sequence sigma, ceil: shangxian
    x = np.arange(-halfsize,halfsize+1)         # sequence from -halfsize to halfsize+1

    if len(sigma) == 1:
        if X is None:
            X = x
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) )
    if len(sigma) == 2:
        if X is None or Y is None:
            [X,Y] =np.meshgrid(x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) )
    if len(sigma) == 3:
        if X is None or Y is None or Z is None:
            [X,Y,Z] =np.meshgrid(x,x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) ) * np.exp( (-1)*np.power(Z,2)/(2 * np.power(sigma[2],2)))
    if len(sigma) > 3:
        print ('Only support up to dimension 3')
    
    return k /np.sum(np.abs(k))

def gradCompute(img, dogsigma):
    dogkercc = doggen([dogsigma, dogsigma, dogsigma])
    dogkerrr = np.transpose( dogkercc, (1, 0, 2) )
    dogkerzz = np.transpose( dogkercc, (0, 2, 1) )

    gcc = signal.convolve(img, dogkercc, mode = 'valid')
    grr = signal.convolve(img, dogkerrr, mode = 'valid')
    gzz = signal.convolve(img, dogkerzz, mode = 'valid')

    # Gradient products
    gp = type('', (), {})()
    gp.gprrrr = grr * grr
    gp.gprrcc = grr * gcc
    gp.gprrzz = grr * gzz
    gp.gpcccc = gcc * gcc
    gp.gpcczz = gcc * gzz
    gp.gpzzzz = gzz * gzz

    # Gradient amplitude
    ga = np.sqrt(gp.gprrrr + gp.gpcccc + gp.gpzzzz)

    return ga, gp



## the optimal patch_size to retrieve the exact size of CLARITY as the originally synthetic one!! do not change this anymore!!

In [12]:
patch_size = 6.6

In [13]:

img_stack = nib.load('syn_CLARITY.nii.gz')
imgstack = img_stack.get_fdata()/255.0


# select sigma
dogsigma = [0.6]
vox_size = [15, 15, 15] #um
#define the voxel dimension to apply tensor
voxel_dim = np.array([patch_size,patch_size,patch_size]) # x,y,z in voxel dimension
volume_dim = (np.array(imgstack.shape)/voxel_dim).astype(np.int) # in voxel dimesion: removed voxel at the border

fsl_tensor = np.zeros((volume_dim[0], volume_dim[1], volume_dim[2], 6))
for x in range (0,volume_dim[0]):
    for y in range (0,volume_dim[1]):
        for z in range (0, volume_dim[2]):
            [ga,gp] = gradCompute(imgstack[int(x*voxel_dim[0]):int((x+1)*voxel_dim[0]), int(y*voxel_dim[1]):int((y+1)*voxel_dim[1]), int(z*voxel_dim[2]):int((z+1)*voxel_dim[2])], dogsigma)

            lowthreh = np.percentile(ga.reshape(-1), 20)
            highthreh = np.percentile(ga.reshape(-1), 80)

            mask = np.ones(ga.shape)
            mask[ga  < lowthreh] = 0
            mask[ga  > highthreh] = 0

            gprrrr = np.sum(gp.gprrrr[mask==1])
            gprrcc = np.sum(gp.gprrcc[mask==1])
            gprrzz = np.sum(gp.gprrzz[mask==1])
            gpcccc = np.sum(gp.gpcccc[mask==1])
            gpcczz = np.sum(gp.gpcczz[mask==1])
            gpzzzz = np.sum(gp.gpzzzz[mask==1])

            fsl_tensor[x,y,z] = np.array([[gprrrr, gprrcc, gprrzz,gpcccc, gpcczz, gpzzzz]])
affine = np.eye(4)
affine[0,0]=vox_size[0]*voxel_dim[0]*0.001
affine[1,1]=vox_size[1]*voxel_dim[1]*0.001
affine[2,2]=vox_size[2]*voxel_dim[2]*0.001
img_fsl_tensor = nib.Nifti1Image(fsl_tensor, affine)
nib.save(img_fsl_tensor, 'fsltensor_ftrial.nii.gz')
CMD='fsl5.0-fslmaths fsltensor_ftrial.nii.gz -tensor_decomp fsltensor_ftrial'
subprocess.call(CMD, shell=True)

/home/zhiwei/.local/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


0

## generate the mask for subsequent tractography generation

In [14]:
data = nib.load('fsltensor_ftrial_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask_syn.nii')

##  generate the tractography and subsequent CLARITY based on the synthetic CLARITY images

In [15]:
number = 100000

In [16]:
CMD='tckgen fsltensor_ftrial_V3.nii.gz before0.015.tck -algorithm FACT -seed_image mask_syn.nii -mask mask_syn.nii -select 100000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [17]:
CMD = 'TractConverter.py -i before0.015.tck -o before0.015.trk -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [18]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk')

In [19]:
for items in range(len(fib_original)):
    fib_before.append(fib_original[items])
nib.trackvis.write('fibers_before_all.trk',fib_before,hdr_before)

In [20]:
CMD = 'TractConverter.py -i fibers_before_all.trk -o fibers_before_all.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [21]:
CMD='tckmap fibers_before_all.tck syn_CLARITY_before.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## function for comparison of two tractography on a voxel base

In [22]:
def compClarity(volume1,volume2):
    error = 0.0
    for x in range(volume1.shape[0]):
        for y in range(volume1.shape[1]):
            for z in range(volume1.shape[2]):
                error = error + np.abs(volume1[x,y,z]-volume2[x,y,z])
    return error

## apply COMMIT to the generated tractography

In [23]:
path_commit = '/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT'
path = '/home/zhiwei/Desktop/semester-projects/results_script1'

In [24]:
trk_file_before = '/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before_all.trk'

In [25]:
from commit import trk2dictionary
trk2dictionary.run(filename_trk   = trk_file_before ,path_out= path_commit )
import commit
commit.core.setup()


-> Creating the dictionary from tractogram:
	* Segment position = COMPUTE INTERSECTIONS
	* Fiber shift X    = 0.000 (voxel-size units)
	* Fiber shift Y    = 0.000 (voxel-size units)
	* Fiber shift Z    = 0.000 (voxel-size units)
	* Points to skip   = 0
	* Min segment len  = 1.00e-03
	* Do not blur fibers
	* Loading data:
		* tractogram
			- 28 x 26 x 5
			- 0.0990 x 0.0990 x 0.0990
			- 103000 fibers
		* no mask specified to filter IC compartments
		* no dataset specified for EC compartments
		* output written to "/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT"
	* Generate tractogram matching the dictionary: 
	  [ 102756 fibers kept ]
   [ 15.5 seconds ]


In [26]:
mit=commit.Evaluation('.','results_script1')

In [27]:
mit.load_data('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY.nii.gz','/home/zhiwei/Desktop/semester-projects/results_script1/BVECTOR.scheme')
mit.set_model('VolumeFractions')
hasISO=False
mit.model.set(hasISO)
mit.generate_kernels(regenerate=True)


-> Loading data:
	* DWI signal...
		- dim    = 191 x 178 x 39 x 1
		- pixdim = 0.015 x 0.015 x 0.015
	* Acquisition scheme...
		- 1 samples, 1 shells
		- 0 @ b=0 , 1 @ b=1000.0
   [ 0.0 seconds ]

-> Preprocessing:
	* There are no b0 volume(s) for normalization... [ min=0.00,  mean=0.26, max=27.00 ]
   [ 0.0 seconds ]

-> Simulating with "Volume fractions" model:
   [ 0.0 seconds ]


In [28]:
mit.load_kernels()
mit.load_dictionary(path_commit)
mit.set_threads(1)


-> Resampling LUT for subject "results_script1":
	* Keeping all b0 volume(s)... [ OK ]
	* Normalizing... [ OK ]
   [ 0.1 seconds ]

-> Loading the dictionary:
	* segments from the tracts... [ 102756 fibers and 5062543 segments ]
	* segments from the peaks...  [ 0 segments ]
	* isotropic contributions...  [ 3591 voxels ]
	* post-processing...          [ OK ]
   [ 1.9 seconds ]

-> Distributing workload to different threads:
	* number of threads : 1
	* A operator...  [ OK ]
	* A' operator... [ OK ]
   [ 0.0 seconds ]


In [29]:
mit.build_operator()


-> Building linear operator A:
   [ 4.4 seconds ]


In [30]:
mit.fit(max_iter = 100000000)
## mit.fit(tol_fun = 1e-7, max_iter = 100000000)


-> Fit model

      |     ||Ax-y||     |  Cost function    Abs error      Rel error    |     Abs x          Rel x
------|------------------|-----------------------------------------------|------------------------------
   1  |   3.3978824e+03  |  5.7728025e+06  4.2028779e+07  7.2804811e+00  |  1.9580074e+02  1.0962577e+00
   2  |   2.2944820e+03  |  2.6323239e+06  3.1404786e+06  1.1930442e+00  |  4.9795691e+01  3.5772116e-01
   3  |   1.5742375e+03  |  1.2391119e+06  1.3932120e+06  1.1243633e+00  |  3.7264291e+01  3.4042360e-01
   4  |   1.0829457e+03  |  5.8638573e+05  6.5272616e+05  1.1131344e+00  |  2.8597968e+01  3.2786143e-01
   5  |   7.5722276e+02  |  2.8669315e+05  2.9969258e+05  1.0453427e+00  |  2.1533404e+01  3.0354126e-01
   6  |   5.4605217e+02  |  1.4908649e+05  1.3760666e+05  9.2299888e-01  |  1.6051959e+01  2.7218167e-01
   7  |   4.0503618e+02  |  8.2027154e+04  6.7059335e+04  8.1752606e-01  |  1.2254760e+01  2.4508493e-01
   8  |   3.0922513e+02  |  4.7810089e+04  3.

  77  |   1.2135917e+01  |  7.3640246e+01  3.3692593e-01  4.5752962e-03  |  8.6003260e-02  1.8691180e-02
  78  |   1.2109355e+01  |  7.3318237e+01  3.2200858e-01  4.3919302e-03  |  8.4650719e-02  1.8347183e-02
  79  |   1.2083889e+01  |  7.3010189e+01  3.0804747e-01  4.2192394e-03  |  8.3165922e-02  1.7971127e-02
  80  |   1.2059609e+01  |  7.2717080e+01  2.9310950e-01  4.0308205e-03  |  8.1221055e-02  1.7493425e-02
  81  |   1.2036396e+01  |  7.2437416e+01  2.7966405e-01  3.8607679e-03  |  7.9780630e-02  1.7122858e-02
  82  |   1.2013989e+01  |  7.2167967e+01  2.6944887e-01  3.7336353e-03  |  7.8658962e-02  1.6819167e-02
  83  |   1.1992340e+01  |  7.1908115e+01  2.5985194e-01  3.6136663e-03  |  7.7399420e-02  1.6484829e-02
  84  |   1.1971372e+01  |  7.1656878e+01  2.5123670e-01  3.5061072e-03  |  7.6375724e-02  1.6199927e-02
  85  |   1.1950994e+01  |  7.1413126e+01  2.4375264e-01  3.4132750e-03  |  7.5569489e-02  1.5960354e-02
  86  |   1.1931290e+01  |  7.1177838e+01  2.3528804e-0

In [31]:
mit.save_results(save_coeff=True)


-> Saving results to "Results_VolumeFractions/*":
	* configuration and results:
		- pickle...  [ OK ]
		- txt...  [ OK ]
	* fitting errors:
		- RMSE...  [ 0.063 +/- 0.182 ]
		- NRMSE... [ 0.019 +/- 0.111 ]
	* voxelwise contributions:
		- intra-axonal [ OK ]
		- extra-axonal [ OK ]
		- isotropic    [ OK ]
   [ 0.9 seconds ]


## start change

In [266]:
trk_file = os.path.join(path_commit,'dictionary_TRK_fibers.trk')
fib_tmp,trk_hdr_tmp=nib.trackvis.read(trk_file)

## generate tractography after applying COMMIT with weight > 0, and get fib_after,hdr_after

In [267]:
f = open('/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT/Results_VolumeFractions/xic.txt')


In [268]:
import heapq
data = f.read().splitlines()
data = map(eval,data)
max_index_list = map(data.index, heapq.nlargest(2089,data))

#j=01
#for items in range(len(fib_tmp)-1,-1,-1):
#    if data[j] == 0:
#        del fib_tmp[items]
#    j=j+1
#nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [269]:
j=0
for items in range(len(fib_tmp)-1,-1,-1):
    if items not in max_index_list:
        del fib_tmp[items]
nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [270]:
fib_after,hdr_after=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')

In [271]:
len(fib_after)

2089

In [272]:
len(data)-data.count(0)

2089

In [273]:
CMD = 'TractConverter.py -i fibers_after.trk -o fibers_after.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [274]:
CMD='tckmap fibers_after.tck syn_CLARITY_after.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## downsample fibers(before/after) to the original one, take mean distance and std as measurement

## downsample the fibers and get tracks_before, tracks_after, tracks_original

In [275]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before_all.trk')

In [276]:
from dipy.tracking.metrics import downsample
import random
tracks_after = [downsample(fib_after[i][0],7) for i in range(len(fib_after))]
tracks_original = [downsample(fib_original[i][0],7) for i in range(len(fib_original))]
## generate a random list to get equal number of fibers from fib before COMMIT
list = range(len(fib_before))
slice2 = random.sample(list, len(fib_after))
tracks_before = [downsample(fib_before[i][0],7) for i in slice2]

In [277]:
## generate a random list to get equal number of fibers from fib before COMMIT
number_of_trials=20
list = range(len(fib_before))
slice1 = np.zeros((number_of_trials,len(fib_after)),dtype=int)
tracks_before1 = np.zeros((number_of_trials,len(fib_after),7,3))
DM_before1 = np.zeros((number_of_trials,len(fib_after),3000))
minD_before1 = np.zeros((number_of_trials,len(fib_after)))
for i in range(number_of_trials):
    slice1[i] = random.sample(list, len(fib_after))
    tracks_before1[i] = [downsample(fib_before[j][0],7) for j in slice1[i]]
    DM_before1[i] = bundles_distances_mdf(tracks_before1[i], tracks_original)
    minD_before1[i] = [min(DM_before1[i,j,:]) for j in range(len(DM_before1[i]))]

In [ ]:
meanD_before = np.zeros(number_of_trials)
stdD_before = np.zeros(number_of_trials)
for i in range(number_of_trials):
    meanD_before[i]= np.mean(minD_before1[i])
    stdD_before[i]= np.std(minD_before1[i])


In [278]:
DM_before1.shape

(2089, 3000)

## compare tracks_after and tracks_before with tracks_original with mean distance and STD

In [279]:
meanD_before = np.zeros(number_of_trials)
stdD_before = np.zeros(number_of_trials)
for i in range(number_of_trials):
    meanD_before[i]= np.mean(minD_before1[i])
    stdD_before[i]= np.std(minD_before1[i])


In [280]:
meanD_before_all = np.mean(meanD_before)
stdD_before_all = np.mean(stdD_before)
meanD_after= np.mean(minD_after)
stdD_after= np.std(minD_after)

0.1379378595130403

In [ ]:
meanD_after

In [281]:
meanD_before_all

0.19356694179393133

## Compare on a voxel level

original resolution -- 0.015mm

In [282]:
for items in range(len(fib_before)-1,-1,-1):
    if items not in slice2:
        del fib_before[items]
    j=j+1
nib.trackvis.write('fibers_before.trk',fib_before,hdr_before)

In [283]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

In [284]:
len(fib_before)

2089

In [285]:
CMD = 'TractConverter.py -i fibers_before.trk -o fibers_before.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [286]:
CMD='tckmap fibers_before.tck syn_CLARITY_random.nii.gz -vox 0.015' 
subprocess.call(CMD, shell = True)

0

In [287]:
data_original = nib.load('syn_CLARITY.nii.gz')
data_before = nib.load('syn_CLARITY_random.nii.gz')
data_after = nib.load('syn_CLARITY_after.nii.gz')
original = data_original.get_fdata()
random = data_before.get_fdata()
after = data_after.get_fdata()

## normalization
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))
volume_original = data_original.get_fdata()/sum_original
volume_before = data_before.get_fdata()/sum_before
volume_after = data_after.get_fdata()/sum_after

## correlation measure to conduct grey level matching

center_original = (original-sum_original/(191*178*39)).flatten()
center_before = (random - sum_before/(191*178*39)).flatten()
center_after = (after - sum_after/(191*178*39)).flatten()

module_original = np.sqrt(np.dot(center_original,center_original))
module_before = np.sqrt(np.dot(center_before,center_before))
module_after = np.sqrt(np.dot(center_after,center_after))

random_coef = np.dot(center_original.flatten(),center_before.flatten())/(module_original*module_before)
COMMIT_coef = np.dot(center_original.flatten(),center_after.flatten())/(module_original*module_after)

print(random_coef)
print(COMMIT_coef)

In [288]:
square_error_before = compClarity(volume_before, volume_original)
square_error_after = compClarity(volume_after, volume_original)

In [289]:
print(square_error_before)
print(square_error_after)

1.7092299958069348
1.7864137696315068


## change the resolution here

resolution based on mean distance after COMMIT

CMD='tckmap origin0.015.tck syn_CLARITY_COMMIT_res.nii.gz -vox 0.195' 
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_after.tck syn_CLARITY_after_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_before.tck syn_CLARITY_random_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

data_original_COMMIT_res = nib.load('syn_CLARITY_COMMIT_res.nii.gz')
data_before_COMMIT_res = nib.load('syn_CLARITY_random_COMMIT_res.nii.gz')
data_after_COMMIT_res = nib.load('syn_CLARITY_after_COMMIT_res.nii.gz')
## normalization
sum_original_COMMIT_res = sum(sum(sum(data_original_COMMIT_res.get_fdata())))
sum_before_COMMIT_res = sum(sum(sum(data_before_COMMIT_res.get_fdata())))
sum_after_COMMIT_res = sum(sum(sum(data_after_COMMIT_res.get_fdata())))
volume_original_COMMIT_res = data_original_COMMIT_res.get_fdata()/sum_original_COMMIT_res
volume_before_COMMIT_res = data_before_COMMIT_res.get_fdata()/sum_before_COMMIT_res
volume_after_COMMIT_res = data_after_COMMIT_res.get_fdata()/sum_after_COMMIT_res

square_error_before_COMMIT_res = compClarity(volume_before_COMMIT_res, volume_original_COMMIT_res)
square_error_after_COMMIT_res = compClarity(volume_after_COMMIT_res, volume_original_COMMIT_res)

## Compare on a voxel level - binary

original resolution -- 0.015mm

In [290]:
def binarize(volume):
    volume_prime = np.zeros([len(volume),len(volume[1]),len(volume[1][1])])
    for x in range(len(volume)):
        for y in range(len(volume[1])):
            for z in range(len(volume[1][1])):
                if volume[x,y,z]!=0:
                    volume_prime[x,y,z]=1
    return volume_prime

In [291]:
Volume_Original = binarize(volume_original)
Volume_Before = binarize(volume_before)
Volume_After = binarize(volume_after)
error_before = compClarity(Volume_Before, Volume_Original)
error_after = compClarity(Volume_After, Volume_Original)

In [292]:
print(error_before)
print(error_after)

292594.0
248308.0


In [293]:
affine=nib.load('/home/zhiwei/Desktop/semester-projects/results_script1/fsltensor_ftrial_FA.nii.gz')
imgR=nib.Nifti1Image(volume_before,affine.affine)
nib.save(imgR,'RNB')
imgR=nib.Nifti1Image(Volume_Before,affine.affine)
nib.save(imgR,'RB')
imgA=nib.Nifti1Image(volume_after,affine.affine)
nib.save(imgA,'CNB')
imga=nib.Nifti1Image(Volume_After,affine.affine)
nib.save(imga,'CB')
imgGT=nib.Nifti1Image(volume_original,affine.affine)
nib.save(imgA,'GTNB')
imga=nib.Nifti1Image(Volume_Original,affine.affine)
nib.save(imga,'GTB')


## change the resolution here

COMMIT resolution 0.186

Volume_Original_COMMIT_res = binarize(volume_original_COMMIT_res)
Volume_Before_COMMIT_res = binarize(volume_before_COMMIT_res)
Volume_After_COMMIT_res = binarize(volume_after_COMMIT_res)
error_before_COMMIT_res = compClarity(Volume_Before_COMMIT_res, Volume_Original_COMMIT_res)
error_after_COMMIT_res = compClarity(Volume_After_COMMIT_res, Volume_Original_COMMIT_res)

## save results

In [260]:
cd ..

/home/zhiwei/Desktop/semester-projects


In [261]:
f = open('forPPT.txt','a') 
f.write('\n\n\n\n')
f.write('500')
f.write('\n\n    meanD_random is '+ str(meanD_before_all))
f.write('\n    meanD_COMMIT is '+str(meanD_after))
f.write('\n\n    stdD_random is '+str(stdD_before))
f.write('\n    stdD_COMMIT is '+str(stdD_after))
f.write('\n\n    B_error_random is '+str(error_before))
f.write('\n    B_error_COMMIT is '+str(error_after))
f.write('\n\n    NB_error_random is '+str(square_error_before))
f.write('\n    NB_error_COMMIT is '+str(square_error_after))
f.close()

cd results_script1

file_script = 'results1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.tck','results1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk','results1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.tck','results1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY_after.nii.gz','results1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY_random.nii.gz','results1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk','results1')

os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.tck')
os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')
os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.tck')
os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY_after.nii.gz')
os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY_random.nii.gz')
os.remove('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

f = open('VOXEL_B_random_COMMIT.txt','a')
f.write(str(error_before)+', ')
f.write(str(error_after)+', ')
f.close()

f = open('VOXEL_NB_random_commit.txt','a')
f.write(str(square_error_before)+', ')
f.write(str(square_error_after)+', ')
f.close()


f = open('coef_random_COMMIT.txt','a')
f.write(str(random_coef)+', ')
f.write(str(COMMIT_coef)+', ')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res - BINARIZATION:')
f.write('\nerror random: ' + str(error_before_COMMIT_res))
f.write('\nerror COMMIT: ' + str(error_after_COMMIT_res)) 
f.write('\n\n')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res- non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before_COMMIT_res))
f.write('\nsquare error COMMIT: ' + str(square_error_after_COMMIT_res)) 
f.write('\n\n\n\n\n')
f.close()

In [ ]:
os.rename('results_script1','5_PERCENTER_%d'%order)